In [14]:
import albumentations as A
import random

transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.OneOf([
            A.GaussNoise(),
            A.GaussNoise(),
        ], p=0.2),
        A.OneOf([
            A.MotionBlur(p=.2),
            A.MedianBlur(blur_limit=3, p=0.1),
            A.Blur(blur_limit=3, p=0.1),
        ], p=0.1),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=20, p=0.4),
        A.OneOf([
            A.OpticalDistortion(p=0.2),
            A.GridDistortion(p=.1),
        ], p=0.2),
        A.OneOf([
            A.CLAHE(clip_limit=2),
            A.Sharpen(),
            A.Emboss(),
            A.RandomBrightnessContrast(),            
        ], p=0.3),
        A.HueSaturationValue(p=0.2),
    ])

In [18]:
import cv2
import numpy as np
import os
import sys # to access the system

face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

cam = cv2.VideoCapture(0)
cam.set(3, 640) # set video width
cam.set(4, 480) # set video height

face_id = -1
with open('name.txt', mode ='r')as file:
    reader=file.readlines()
    
    if(len(reader)!=0):
        x=reader[len(reader)-1]
        face_id=int(x)+1 #Giving new employee id according to the last id
    else:
        face_id=1

face_id = str(face_id)    

    
print("\n [INFO] Initializing face capture. Look the camera and wait ...")
# Initialize individual sampling face count
count = 0

img = np.array([])

while(True):

    ret, img = cam.read()
    count += 1
    # Press 'ESC' for exiting video
    k = cv2.waitKey(1000) & 0xff
    if k == 27:
        break
    elif count >= 1: # Take 1 face sample and stop video
        break

# Do a bit of cleanup
print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()

# Save the captured image into the img folder
cv2.imwrite("img/" + face_id + ".jpg", img)
print(face_id)

#gray scaling of image
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = face_detector.detectMultiScale(gray, 1.3, 2)

for (x,y,w,h) in faces:

    cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 3)     

        # Save the captured image into the datasets folder
    cv2.imwrite("dataset/User." + str(face_id) + '.' + str(0) + ".jpg", gray[y:y+h,x:x+w])

#For checking if the captured image is proper or not
while True:
    cv2.imshow("image", gray[y:y+h,x:x+w])
    cv2.waitKey(0)
    sys.exit() # to exit from all the processes

cv2.destroyAllWindows() # destroy all windows


 [INFO] Initializing face capture. Look the camera and wait ...

 [INFO] Exiting Program and cleanup stuff
3


SystemExit: 

In [19]:
from tqdm import tqdm 

#Adding the new Id in name.txt file
f = open("name.txt", 'a+')
if(int(face_id)==1):
    f.write(str(0) + '\n')
    f.write(face_id + '\n')
else:
    f.write(face_id + '\n')
f.close()

#Synthesising of data using augmentation 
count=1
for i in tqdm(range(500)):
    augmented_image = transform(image=gray[y:y+h,x:x+w])['image']
    cv2.imwrite("dataset/User." + str(face_id) + '.' + str(count) + ".jpg", augmented_image)
    count=count+1

100%|███████████████████████████████████████████████████████████████████████████████| 500/500 [00:01<00:00, 438.35it/s]
